In [15]:
import sys 
import os
import argparse
sys.path.append('./project/')
sys.path.append(f'/users/Etu2/3701222/.local/lib/python3.7/site-packages')


import pytorch_lightning as pl
from vision_transformer_org import VisionTransformer
from lightning_modules import CIFAR10DataModule, LitClassifierModel, load_from_checkpoint
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor


In [ ]:
! mkdir -p /tmp/vit

In [19]:
directory = "/tmp/vit"
args = argparse.Namespace(
    ##########useful args
    fit=True,
    default_root_dir=f"{directory}/model",
    data_path=f"{directory}/cifar10",
    gpus=-1,
    ########Data Args##########
    image_size=224,
    num_classes=10,
    ########Training Args##########
    learning_rate=1e-4,
    val_size=0.2,
    batch_size=32,
    num_workers=14,
    #######Model args############
    patch_size=32,
    emb_dim=768,
    mlp_dim=3072,
    num_heads=12,
    num_layers=12,
    attn_dropout_rate=0.0,
    dropout_rate=0.1,
    ########Trainer Args##########
    progress_bar_refresh_rate=25,
)

In [22]:
datamodule = CIFAR10DataModule(
        data_dir=args.data_path,
        image_size=args.image_size,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        val_size=args.val_size
)

vit_Backbone = VisionTransformer(
        image_size=(args.image_size, args.image_size),
        patch_size=(args.patch_size, args.patch_size),
        emb_dim=args.emb_dim,
        mlp_dim=args.mlp_dim,
        num_heads=args.num_heads,
        num_layers=args.num_layers,
        num_classes=args.num_classes,
        attn_dropout_rate=args.attn_dropout_rate,
        dropout_rate=args.dropout_rate,
)

checkpoint_callback = ModelCheckpoint(
        monitor='val_acc',
        filename='vit-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}',
        mode='max',
)
lr_monitor = LearningRateMonitor(logging_interval='step')


In [23]:
trainer = pl.Trainer.from_argparse_args(args, callbacks=[checkpoint_callback, lr_monitor])

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [24]:
model = LitClassifierModel(vit_Backbone, **vars(args))

In [25]:
trainer.fit(model, datamodule)

Files already downloaded and verified
Files already downloaded and verified



  | Name     | Type              | Params
-----------------------------------------------
0 | backbone | VisionTransformer | 41.6 K
-----------------------------------------------
41.6 K    Trainable params
0         Non-trainable params
41.6 K    Total params
0.166     Total estimated model params size (MB)


/home/melissa/anaconda3/envs/main/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [ ]:
trainer.test(model, datamodule=datamodule)